### קוד מבוא

In [77]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import adjustText as aT

In [78]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [79]:
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### פונקציות

In [80]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [81]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [82]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [83]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [84]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


In [85]:

def split_index_by_taz(index,taz,min_prec,col_name_to_split):
    index['index_area']=index.area
    
    taz['taz_area']=taz.area

    index_taz=index.overlay(taz[['Taz_num','taz_area','geometry']])

    index_taz['small_area']=index_taz.area

    index_taz['precent_from_big_index']=index_taz['small_area']/index_taz['index_area']
    
    index_taz['precent_from_big_taz']=index_taz['small_area']/index_taz['taz_area']

    index_taz=index_taz.loc[(index_taz['precent_from_big_index']>min_prec)|(index_taz['precent_from_big_taz']>0.9)]
    
    index_taz=index_taz[['id','Taz_num','precent_from_big_index']]

    new_big=index_taz.groupby(['id']).sum()

    index=index.set_index('id')
    index['new_big']=new_big['precent_from_big_index']

    index=pd.merge(index.reset_index(),index_taz,on='id')

    for c in col_name_to_split:
        index['{}'.format(c)]=index['{}'.format(c)]*(index['precent_from_big_index']/index['new_big'])
        
        
    return index

### העלת קבצים

#### קיבולת ברמת יישוב (חלקי) ביו"ש

In [86]:
kibolt_yosh_some_towns=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\מרכיבי_תחזית','230628_Jewish_kibolt_some_towns_in_yosh_till_2050')

In [87]:
col=[ 'Taz_num', 'add_aprt_2020__2025',
 'add_aprt_2025__2030',
 'add_aprt_2030__2035',
 'add_aprt_2035__2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']

In [88]:
kibolt_yosh_some_towns_sum_by_taz=kibolt_yosh_some_towns.pivot_table(index='Taz_num',aggfunc=sum).reset_index()[col]

In [89]:
kibolt_yosh_some_towns_sum_by_taz['kibolet_type']='Shrinkable'

In [90]:
old_col=[ 'Taz_num','add_aprt_2020__2025',
 'add_aprt_2025__2030',
 'add_aprt_2030__2035',
 'add_aprt_2035__2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050','kibolet_type']

new_col=[ 'Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_type']

kibolt_yosh_some_towns_sum_by_taz=kibolt_yosh_some_towns_sum_by_taz[old_col]

kibolt_yosh_some_towns_sum_by_taz.columns=new_col

In [91]:
kibolt_yosh_some_towns_sum_by_taz=kibolt_yosh_some_towns_sum_by_taz.melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

In [92]:
kibolt_yosh_some_towns_sum_by_taz['source']='by_town'

#### קיבולת ברמת אזור תנועה (חלקי)

In [93]:
Jewish_kibolt_some_by_taz=up_load_shp(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\מרכיבי_תחזית\SHP\kibolt_jew_by_some_taz_230628.shp')

In [94]:
old_col=[ 'Taz_num','add_aprt_2',
 'add_aprt_3',
 'add_aprt_4',
 'add_aprt_5',
 'add_aprt_6',
 'add_aprt_7','type_kibol']

In [95]:
new_col=[ 'Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_type']

In [96]:
Jewish_kibolt_some_by_taz=Jewish_kibolt_some_by_taz[old_col]

In [97]:
Jewish_kibolt_some_by_taz.columns=new_col

In [98]:
Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==1,'kibolet_type']='Shrinkable'

In [99]:
Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==2,'kibolet_type']='left_the_station'

In [100]:
Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==3,'kibolet_type']='Depends_on_policy'

In [101]:
Jewish_kibolt_some_by_taz=Jewish_kibolt_some_by_taz.melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

In [102]:
Jewish_kibolt_some_by_taz['source']='by_taz'

#### קיבולת ברמת תכנית עם אזורי תנועה בלבד (חלקי)

In [103]:
Jewish_kibolt_some_by_plan_only_with_taz=up_load_shp(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\מרכיבי_תחזית\SHP\Jewish_kibolt_some_by_plan_with_taz_num.shp')
Jewish_kibolt_some_by_plan_only_with_taz=Jewish_kibolt_some_by_plan_only_with_taz[['Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_ty']].rename(columns={'kibolet_ty':'kibolet_type'})

Jewish_kibolt_some_by_plan_only_with_taz=Jewish_kibolt_some_by_plan_only_with_taz.melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

In [104]:
Jewish_kibolt_some_by_plan_only_with_taz['source']='plan_only_with_taz'

#### קיבולת ברמת תכנית  (חלקי)

In [105]:
df=up_load_df(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\iplan\jew_kibolet\input','index_for_id_polygons_230712')

shp=up_load_shp(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\מרכיבי_תחזית\SHP\230628_id_polygons_for_index.shp')

taz=up_load_shp(r'W:\Model Versions\4.0\TAZ\TAZ_V4_230518_Published.shp')

index=shp.merge(df,on='id',how='outer')

#לחלק את התכניות לפי אזורי תנועה

index=split_index_by_taz(index,taz,0.25,['add_aprt'])

#לייצר קיבולת כל חומש

years=[ '2025', '2030', '2035', '2040', '2045', '2050']

index[years]=index[years].apply(lambda x: x * index['risk_factor']*index['add_aprt'])

col=['Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_type']

Jewish_kibolt_some_by_plan=index[col].fillna(0).melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

C:\Users\gidon\Anaconda3\envs\geo_env\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\gidon\Anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:2196: UserWarning: `keep_geom_type=True` in overlay resulted in 13 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


In [106]:
Jewish_kibolt_some_by_plan['source']='by_plan'

### מכאן  צריך לחבר ביחד את כל הטבלאות

In [107]:
kibolt_jew=pd.concat([Jewish_kibolt_some_by_plan,Jewish_kibolt_some_by_plan_only_with_taz,kibolt_yosh_some_towns_sum_by_taz,Jewish_kibolt_some_by_taz],axis=0)

In [108]:
kibolt_jew['year']=kibolt_jew['year'].astype(str)

In [109]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

### לשנות קיבולת

כדי לשנות קיבולת צריך לעשות את זה לפי מגזר

In [110]:


folder_path=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תרחישי_מלל\מרכיבי_תחזית'

taz_sector_change=up_load_df(folder_path,'230704_taz_num_convert_from_jewish_to_u_orthodox')

lst_change_sector=list(taz_sector_change.Taz_num)


taz.loc[(taz['Taz_num'].isin(lst_change_sector))&(taz['zonetype']=='Judea and Samaria'),'main_secto']='U_Orthodox'

In [111]:
kibolt_jew=kibolt_jew.merge(taz[['Taz_num','main_secto','zonetype']],on='Taz_num',how='left')

שינוי בקיבולת לפי מגזר

In [113]:
kibolt_jew=kibolt_jew.loc[~((kibolt_jew['main_secto']=="Jewish")&
                            (kibolt_jew['zonetype']=="Jerusalem")&
                            (kibolt_jew['kibolet_type']!="left_the_station"))&
                           ~((kibolt_jew['main_secto']=="Jewish")&
                            (kibolt_jew['zonetype']=="Judea and Samaria")&
                            (kibolt_jew['kibolet_type']=="Depends_on_policy"))
                         ]

In [ ]:
kibolt_jew.loc[(kibolt_jew['main_secto']=="Jewish")&
               (kibolt_jew['zonetype']=="Judea and Samaria")&
               (kibolt_jew['kibolet_type']=="Shrinkable"),'add_aprt']=kibolt_jew['add_aprt']*0.65

### ייצוא לקלט של מכונת התפלגות גילים תרחיש דיור

In [75]:
kibolt_jew=kibolt_jew.pivot_table(index='Taz_num',columns='year',values='add_aprt',aggfunc=sum).fillna(0).reset_index()

In [76]:
old_col=[ 'Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050']

new_col=[ 'Taz_num','add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']

kibolt_jew=kibolt_jew[old_col]

kibolt_jew.columns=new_col

save_file=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\iplan\jew_kibolet\output'

kibolt_jew.to_excel('{}\{}_kibolt_jew_till_2050.xlsx'.format(save_file,file_date),index=False)